## Sentinel-2 Multi-Spectral Imager (MSI) Demo

In this tutorial, we will look at MSI data using SatPy. More information about the instrument can be found here: https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-2-msi/msi-instrument

In [ ]:
# Ignore warning and debug messages
# NB: If things go wrong, you can turn the debugger on and turn the warnings filter off.
import warnings
warnings.catch_warnings()
warnings.simplefilter("ignore")
from satpy.utils import debug_off; debug_off()

As before, we can read in the appropriate files using SatPy's utility function:

In [ ]:
from satpy import find_files_and_readers

path_msi = './data/msi/'
files = find_files_and_readers(reader='msi_safe',
                               base_dir=path_msi)

Sentinel-2 MSI files come in the `.SAFE` file format, which is actually series of directories containing geolocated files (.jp2) and metadata (.xml). To view the filenames we can print the files as before:

In [ ]:
# Check the list of files
print(files)

This looks like a bit of a mess but with SatPy's `Scene` object, we can make sense of what we have:

In [ ]:
from satpy import Scene

scn = Scene(files)
scn.available_dataset_names()

Here we can see the 13 spectral channels that MSI has, along with some infromation regarding the sensor geometries. Let's get some more information about a particular band:

In [ ]:
scn.load(['B8A'])
print(scn)

We can also look at the available composite names as before:

In [ ]:
scn.available_composite_names()

Let's see what a true colour image looks like at the native MSI resolution:

In [ ]:
from dask.diagnostics import ProgressBar

scn.load(['true_color'])
lcn = scn.resample(scn.finest_area())

with ProgressBar():
    lcn.save_dataset('true_color', tiled=True, filename='./data/geotiff/msi_tc.tif')

Now, let's resample to the same area that we defined before for MODIS and AHI, so that we can get some appreciation for the differences in spatial resolution among the three sensors:

In [ ]:
from pyresample import create_area_def

# extent = [lonmin, lonmax, latmin, latmax]
extent = [158.6, 159.2, -54.8, -54.4]
area_def = create_area_def('Macquarie Island', "+proj=eqc", units="degrees",
                           area_extent=(extent[0], extent[2], extent[1], extent[3]), resolution=0.0001)
scn.load(['true_color'])
lcn = scn.resample(area_def)

with ProgressBar():
    lcn.save_dataset('true_color', tiled=True, filename='./data/geotiff/msi_mac_island_tc.tif')

As we can see, the MSI data has very high spatial resolution and could be used to study vegetation changes on the island. One comment metric used to assess vegetation health is the Normalised Difference Vegetation Index. To compute the NDVI, we do:

In [ ]:
scn.load(['B04', 'B08'])
lcn = scn.resample(area_def)
# Get the RED and NIR data arrays
red = lcn['B04'].values
nir = lcn['B08'].values
# Compute NDVI
ndvi = (nir-red) / (nir+red)

Finally, we can plot the NDVI using standard matplotlib methods:

In [ ]:
# Plot the data
import matplotlib.pyplot as plt

plt.imshow(ndvi, cmap='YlGn', vmin=0, vmax=1)
plt.show()

## Exercise
Can you compute and plot a different spectral index over Macquarie Island? Some examples are here: https://giscrack.com/list-of-spectral-indices-for-sentinel-and-landsat/